In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

In [ ]:
data.head()

# So this time we are goig to do some cleaning if it is necessary and do some prediction of the quality of the wine based on the feature

Let us know about the dataset

In [ ]:
data.shape

In [ ]:
data.info()

So it seems like there are no missing values in the dataset and now we are all set to training the model

In [ ]:
data.describe()

Let us the correlation between the features

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
x= sns.PairGrid(data)
x = x.map(plt.scatter)

In [ ]:
data.corr()

Let us do some featuring engineering 
As of my research i found the main features which decide the best wine

Sweetness: Wine labels often use the terms “sweet,” “semi-sweet” or “dry.” A dry wine will not be sweet at all.
Acidity: Wines with high acidity will be more tart, whereas low-acidity wines will taste rounder or richer.


Tannin: Tannins are phenolic compounds in the skins of grapes. When tannins are naturally present in the winemaking process or added through aging, the wine will have a more bitter taste. Because tannins also tend to dry out your mouth, people often confuse the tannin level with the “dryness” of a wine, which actually refers to how sweet or not sweet a wine is. The red winemaking process incorporates more tannins, giving some red wines a distinctively dry and bitter finish.


Body: Wines get characterized as having a light body, full body or somewhere in between. The “body” of the wine refers to how heavy or light it feels in your mouth. Generally, red wines have a fuller body than whites, as do wines made from grapes grown in warmer regions, rather than cooler ones.


Alcohol: The higher the percentage of alcohol in your glass of wine, the more it will warm your throat and the back of your mouth. Measured in percentage of alcohol by volume (ABV), most wines contain 11 to 13 percent alcohol, but can range from 5.5 percent all the way up to 20 percent.

From the above info we can consider alcohol content matters most and we have seen in the correleation already.
and next what is intersting is the body of wine.

from another source i foind that

Wines Under 12.5% alcohol (the alcohol percentage should always be written on the wine’s label) are said to light-bodied. These are generally the white wines we think of as crisp and refreshing. Good examples of these wines are Riesling, Italian Prosecco and Vinho Verde.

Wines between 12.5% and 13.5% are considered medium-bodied. Good examples of these wines are Rose, French Burgundy, Pinot Grigio and Sauvignon Blanc.

Finally, any wine over 13.5% alcohol is considered full-bodied. Some wines that are normally over this alcohol level and considered full-bodied are Zinfandel, Syrah/Shiraz, Cabernet, Merlot and Malbec. While the majority of wines over 13.5% alcohol are usually red, Chardonnay is a great example of a white that often can also be considered full-bodied.

Alcohol percent < 12.5 => light-bodied

Alcohol percent 12.5 to 13.5 => medium-bodied

Alcohol percent > 13.5  => full-bodied

In [ ]:
print(data[data['alcohol']<12.5].shape)
print(data[(data['alcohol']>=12.5) & (data['alcohol']<=13.5)].shape)
print(data[data['alcohol']>13.5].shape)

from the above results we can cleary see 
there are:-
    
1508 ligth bodied 

78  medium bodied

12  full bodied

In [ ]:

data.loc[data['alcohol']<12.5, 'body'] = 'Light body' 
data.loc[data['alcohol']>13.5, 'body'] = 'Full body' 
data.loc[(data['alcohol']>=12.5)& (data['alcohol']<=13.5),'body']='Half body' 

In [ ]:
data.head()

Now let us hot encode the body column . i will be using the so called manual method , i will replace the values 



In [ ]:
data['body'] = data['body'].replace(to_replace='Light body' , value =0)
data['body'] = data['body'].replace(to_replace='Half body' , value =1)
data['body'] = data['body'].replace(to_replace='Full body' , value =2)


In [ ]:
data.info()

In [ ]:
data['body'] = data['body'].astype('category')

In [ ]:
data.describe()

In [ ]:
data.corr()

In [ ]:
X = data.drop(columns=['quality'])
y = data['quality']

In [ ]:
from  sklearn.model_selection import train_test_split,cross_val_score

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=47)

In [ ]:
 from sklearn.ensemble import RandomForestClassifier
    
rf1 = RandomForestClassifier(n_jobs=200)

rf1.fit(X_train,y_train)

In [ ]:
rf1.score(X_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
pred_rfc = rf1.predict(X_test)
print(classification_report(y_test, pred_rfc))

Well using randomForest classifier we got arounf 73 percent of accuracy on test set